In [1]:
import yfinance as yf

btc_cad = yf.Ticker("BTC-CAD")
print(btc_cad)

yfinance.Ticker object <BTC-CAD>


Each column represents a different feature, for example 'High', 'Low' and 'Close' prices are all features. These features are also known as attributes, input, or predictor variables which can be used to ascertain a stock price prediction (called labels, output, or target variables).

In [2]:
# get stock info
btc_cad.info

{'companyOfficers': [],
 'name': 'Bitcoin',
 'startDate': 1367107200,
 'description': 'Bitcoin (BTC) is a cryptocurrency . Users are able to generate BTC through the process of mining. Bitcoin has a current supply of 18,686,900. The last known price of Bitcoin is 55,886.08752366 USD and is down -2.64 over the last 24 hours. It is currently trading on 9597 active market(s) with $68,782,161,116.51 traded over the last 24 hours. More information can be found at https://bitcoin.org/.',
 'maxAge': 1,
 'previousClose': 69859.91,
 'regularMarketOpen': 69859.91,
 'twoHundredDayAverage': 43283.41,
 'trailingAnnualDividendYield': None,
 'payoutRatio': None,
 'volume24Hr': 82251292672,
 'regularMarketDayHigh': 70640.37,
 'navPrice': None,
 'averageDailyVolume10Day': 84926071730,
 'totalAssets': None,
 'regularMarketPreviousClose': 69859.91,
 'fiftyDayAverage': 71173.03,
 'trailingAnnualDividendRate': None,
 'open': 69859.91,
 'toCurrency': 'CAD=X',
 'averageVolume10days': 84926071730,
 'expireDat

Retrieve historical market data

In [3]:
# get historical market data, here max is set to 1 year and 7 months.
btc_cad.history(period="max")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-09-18,13570.838867,13664.313477,13552.004883,13561.018555,21500923601,0,0
2019-09-19,13564.007812,13649.998047,13061.373047,13611.212891,26433391055,0,0
2019-09-20,13611.084961,13642.738281,13438.896484,13504.491211,19542795567,0,0
2019-09-21,13507.153320,13513.054688,13264.508789,13289.721680,17806696605,0,0
2019-09-22,13295.555664,13368.072266,13166.497070,13362.695312,17515000629,0,0
...,...,...,...,...,...,...,...
2021-04-16,79368.234375,79831.468750,75262.203125,77018.320312,105437908391,0,0
2021-04-17,76964.703125,78268.406250,75503.000000,75906.359375,82729666943,0,0
2021-04-18,75928.953125,76373.718750,66081.828125,70374.906250,122017577989,0,0


There isn't much data to work from, but I would like to create a dataframe object from the price data and try and use it to forecast future price information.
# Import Libraries
The bulk of the libraries I will use are as follows. This may change as I progress.

In [4]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn import preprocessing, svm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

Creating a dataframe to work from:

In [5]:
bitcoin = btc_cad.history(period="max")
print(bitcoin.head())

                    Open          High           Low         Close  \
Date                                                                 
2019-09-18  13570.838867  13664.313477  13552.004883  13561.018555   
2019-09-19  13564.007812  13649.998047  13061.373047  13611.212891   
2019-09-20  13611.084961  13642.738281  13438.896484  13504.491211   
2019-09-21  13507.153320  13513.054688  13264.508789  13289.721680   
2019-09-22  13295.555664  13368.072266  13166.497070  13362.695312   

                 Volume  Dividends  Stock Splits  
Date                                              
2019-09-18  21500923601          0             0  
2019-09-19  26433391055          0             0  
2019-09-20  19542795567          0             0  
2019-09-21  17806696605          0             0  
2019-09-22  17515000629          0             0  


And now the tail of the dataframe.

In [6]:
print(bitcoin.tail())

                    Open          High           Low         Close  \
Date                                                                 
2021-04-16  79368.234375  79831.468750  75262.203125  77018.320312   
2021-04-17  76964.703125  78268.406250  75503.000000  75906.359375   
2021-04-18  75928.953125  76373.718750  66081.828125  70374.906250   
2021-04-19  70344.109375  71803.609375  68107.265625  69788.234375   
2021-04-20  69859.906250  70640.367188  66774.476562  69610.000000   

                  Volume  Dividends  Stock Splits  
Date                                               
2021-04-16  105437908391          0             0  
2021-04-17   82729666943          0             0  
2021-04-18  122017577989          0             0  
2021-04-19   81836928952          0             0  
2021-04-20   82912714752          0             0  


Next...Looking into a statistical summary using the describe() method:

In [7]:
bitcoin.describe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
count,577.000000,577.000000,577.000000,577.000000,5.770000e+02,577.0,577.0
mean,22668.768844,23280.556739,22052.326714,22760.611188,4.799133e+10,0.0,0.0
std,19522.291806,20164.169190,18831.559749,19611.430088,2.874595e+10,0.0,0.0
min,6996.441895,7438.541992,5720.531250,6930.849121,1.624637e+10,0.0,0.0
25%,11337.793945,11537.156250,11083.945312,11343.078125,2.858304e+10,0.0,0.0
50%,13232.760742,13478.236328,12960.170898,13233.329102,4.225350e+10,0.0,0.0
75%,23601.916016,24416.914062,23047.320312,23800.703125,6.119120e+10,0.0,0.0
max,79623.851562,81445.671875,77726.054688,79598.406250,4.470696e+11,0.0,0.0


So I can see there are 577 observations or records in this dataframe, but in order to confirm this I can call the count method():

In [9]:
bitcoin.count()

Open            577
High            577
Low             577
Close           577
Volume          577
Dividends       577
Stock Splits    577
dtype: int64